### Классифиция комментариев на позитивные и негативные

**Описание проекта**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок. Постройте модель со значением метрики качества F1 не меньше 0.75.

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
!pip install transformers
!pip install torch
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-28 17:59:47.418592: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 17:59:48.875331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-28 17:59:50.293208

In [3]:
import pandas as pd
import numpy as np
import spacy
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import transformers
from tqdm import notebook 
from sklearn.metrics import f1_score
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertModel, BertForMaskedLM
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

##  Загрузите и подготовьте данные. 

In [4]:
link = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'
try:
  df = pd.read_csv(link,  sep=',', index_col=0)
except:
  df = pd.read_csv('/datasets/toxic_comments.csv', sep=',', index_col=0)

In [5]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [7]:
df.duplicated().sum()

0

Данные загружены, проверены на наличие дубликатов, просмотрена общая информация о них. 
Дубликатов нет, типы данных соответствующий. 

Приступим к подготовке данных

Инициализируем токенизатор как объект класса BertTokenizer, объявим BERT

In [8]:
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")
model = BertModel.from_pretrained("unitary/toxic-bert")  

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
#обрезаем под нужное кол-во токенов
tokenized = df['text'].apply(lambda x: tokenizer.encode(x, max_length=128, truncation=True, add_special_tokens=True)) 
#создаем padded и attention_mas и добавим нули
padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True)
attention_mask = padded > 0
attention_mask = attention_mask.type(torch.LongTensor) 

In [10]:
# датасет и создадим загрузчик с батчом 32
dataset = TensorDataset(attention_mask, padded)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0) 

# проверяем доступно ли нам GPU (работаем в Colab)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(f'Device: {device}')

Device: cuda:0


In [11]:
# получение эмбеддингов с помощью GPU
embeddings = []
model.to(device)
model.eval() 
for attention_mask, padded in notebook.tqdm(dataloader):
    attention_mask, padded = attention_mask.to(device), padded.to(device)

    with torch.no_grad(): 
        batch_embeddings = model(padded, attention_mask=attention_mask)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) 
#Соберём все эмбеддинги в матрицу признаков
features = np.concatenate(embeddings)

  0%|          | 0/4978 [00:00<?, ?it/s]

In [12]:
# Объявляем таргет и разделяем выборки для обучения
target = df['toxic']
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.3, random_state=12345)

С помщью toxic-bert были обработаны текстовые данные и созданы признаки для обучения. Данные были поделены на выборки.

## Обучите разные модели.

Построим и подберем гиперпараметры для моделей LogisticRegression, CatBoostClassifier, RandomForestClassifier. Выберем лучшую модель для тестирования.

In [13]:
# Строим LogisticRegression
params_lr = {
    'solver': ['lbfgs', 'liblinear'],
    'C': [i for i in range(1, 10)]
}
model_lr =  RandomizedSearchCV(LogisticRegression(random_state=12345, max_iter = 200), params_lr,  n_iter=10, scoring='f1', random_state=12345, n_jobs=-1, cv=3, verbose=10)
model_lr.fit(features_train, target_train)
# Определяем лучяший результат и его гиперпараметры
display(model_lr.best_params_)
display(model_lr.best_score_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'solver': 'lbfgs', 'C': 1}

-0.9326409824396994

Для логистической регресси с параметрами solver = 'lbfgs', C = 1 лучший результат получился 
F1 = 0.9326409824396994

In [19]:
# Строим CatBoostClassifier
params_cbc = {
    'learning_rate': [0.01, 0.1],
    'depth': [None] + [i for i in range(2, 12)],
}
model_cbc = RandomizedSearchCV(CatBoostClassifier(random_state=12345, iterations=100), params_cbc, n_iter=10, scoring='f1', random_state=12345, n_jobs=-1, cv=3, verbose=10)

# Обучаем CatBoostClassifier
model_cbc.fit(features_train, target_train)
# Определяем лучяший результат и его гиперпараметры
display(model_cbc.best_params_)
display(model_cbc.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
0:	learn: 0.4665321	total: 480ms	remaining: 47.5s
1:	learn: 0.3082468	total: 831ms	remaining: 40.7s
2:	learn: 0.2120066	total: 1.24s	remaining: 40.1s
3:	learn: 0.1536604	total: 1.62s	remaining: 38.8s
4:	learn: 0.1161114	total: 2.01s	remaining: 38.2s
5:	learn: 0.0941202	total: 2.39s	remaining: 37.5s
6:	learn: 0.0789047	total: 2.75s	remaining: 36.5s
7:	learn: 0.0691861	total: 3.1s	remaining: 35.6s
8:	learn: 0.0622195	total: 3.48s	remaining: 35.2s
9:	learn: 0.0571417	total: 3.85s	remaining: 34.6s
10:	learn: 0.0536526	total: 4.18s	remaining: 33.9s
11:	learn: 0.0506564	total: 4.56s	remaining: 33.4s
12:	learn: 0.0485887	total: 4.91s	remaining: 32.9s
13:	learn: 0.0469279	total: 5.28s	remaining: 32.5s
14:	learn: 0.0456929	total: 5.73s	remaining: 32.5s
15:	learn: 0.0446805	total: 6.08s	remaining: 31.9s
16:	learn: 0.0438867	total: 6.49s	remaining: 31.7s
17:	learn: 0.0431728	total: 6.83s	remaining: 31.1s
18:	learn: 0.0426264	total: 7.18

{'learning_rate': 0.1, 'depth': 6}

-0.9329580937119072

Для CatBoostClassifier с гиперпараметрами learning_rate = 0.1, depth = 6 лучший результат получился F1 = 0.9329580937119072

In [20]:
# Строим RandomForestClassifier

params_rfc = {  
    'n_estimators': [i for i in range(30, 90)],
    'max_depth' : [None] + [i for i in range(2, 12)],
}

model_rfc = RandomizedSearchCV(RandomForestClassifier(random_state=12345), params_rfc, n_iter=10, scoring='f1', random_state=12345, n_jobs=-1, cv=3, verbose=10)

# Обучаем RandomForestClassifier
model_rfc.fit(features_train, target_train)
# Определяем лучяший результат и его гиперпараметры
display(model_rfc.best_params_)
display(model_rfc.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


{'n_estimators': 87, 'max_depth': 7}

0.9339736537488109

Для RandomForestClassifier с гиперпараметрами n_estimators = 87, max_depth = 7 лучший результат получился F1 =0.9339736537488109

Итог: Лучший результат показала модель Случайного леса, тестирование проведем на ней.

In [27]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators = 87, max_depth = 7)
model_rfc.fit(features_train, target_train)

RandomForestClassifier(max_depth=7, n_estimators=87, random_state=12345)

In [28]:
predictions = model_rfc.predict(features_test)
print('F1:', f1_score(target_test, predictions))

F1: 0.9400267462195246


На тестовой выборке модель RandomForestClassifier показала F1 = 0.94

## Сделайте выводы


Была найдена необходимая модель, метрика F1 на тестовой выборке удовлетворяет условию. Лучшая модель RandomForestClassifier, метрика  = 0,94.

Обработка и подготовка текстов блягодаря BERT  производится качественнее, метрики очень высокие, однако это очень ресурсозатратный процесс, который требует дополнительных мощностей и времени.
